In [14]:
from azure.data.tables import TableServiceClient

In [16]:
type(tmp)

NameError: name 'tmp' is not defined

In [ ]:
from contextlib import contextmanager

@contextmanager
def getTable(tableName: str):
    try:
        with TableServiceClient.from_connection_string(conn_str=ConStr) as client:
            yield client.create_table_if_not_exists(table_name=tableName)
    finally:
        pass

In [ ]:
entity1 = {"PartitionKey": "pk001", "RowKey": "rk001", "Value": 4, "day": "Monday", "float": 4.003}
entity2 = {"PartitionKey": "pk001", "RowKey": "rk002", "Value": 4, "day": "Tuesday", "float": 4.003}
entity3 = {"PartitionKey": "pk001", "RowKey": "rk003", "Value": 4, "day": "Wednesday", "float": 4.003}
entity4 = {"PartitionKey": "pk001", "RowKey": "rk004", "Value": 4, "day": "Thursday", "float": 4.003}

# Instantiate a TableClient using a connection string
with getTable('SomeRandomTable') as table_client:

    operations = [
        ("create", entity1),
        # ("delete", entity2),
        ("upsert", entity3),
        # ("update", entity4, {"mode": "replace"}),
    ]
    try:
        table_client.submit_transaction(operations)
    except Exception as e:
        print("There was an error with the transaction operation")
        print(f"Error: {e}")
    # [END batching]

In [ ]:
import pandas as pd

In [ ]:
df1= pd.read_csv('src/Models/PipelineMetaDatahch.csv')
df1.columns

In [ ]:
import duckdb


In [ ]:
df1

In [ ]:
with duckdb.connect('src/Models/EnbridgeMeta.db') as con:
    con.execute("""
        CREATE TABLE IF NOT EXISTS EnbridgeSegmentsTable AS SELECT * FROM df1;
    """)

In [ ]:
df2= pd.read_csv('src/Models/MetaData.csv')
df2.columns

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df1['LocZone_Del'].unique()

In [ ]:
df2['Loc Zone'].unique()

In [ ]:
{
    'EffDate' : 'Eff Date',
    'DirFlow' : 'Dir Flow',
    'Loc' : 'Loc',
    'LocCnty' : 'Loc Cnty',
    'LocName' : 'Loc Name',
    'LocStAbbrev' : 'Loc St Abbrev',
    'LocTypeInd' : 'Loc Type Ind',
    'LocZone_Del' : '',
    
    
    
    'LineNom' : 'Line Nom',
    'LineType' : 'Line Type',
    'PipeGrade' : 'Pipe Grade',
    'SpecCoat' : 'Spec Coat',
    'YearBuilt' : 'Year Built', 
    
}



In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('src/Models/OA_Processed.csv')

In [ ]:
df.info()

In [ ]:
df['TSP'].apply(lambda x: len(str(x))).max()

In [ ]:
df['GFLOC'] = df['GFPipeID'].astype(str)+ df['Loc'].apply(lambda x: x.rjust(6,'0'))

In [ ]:
df['GFLOC']

In [ ]:
df['Loc']

In [1]:
import polars as pl
from pathlib import Path
import functools
import operator
from src.artifacts import getPipes
from src.enbridgescrape import formatOA, metaMunge
from src.artifacts import getPipes

In [2]:
df = formatOA()

/Users/saijagadeeshyadavalli/Coding/Particles/GasFundies/enbridgeGitDump/src/artifacts/ModelKeeper.py:60: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(data=con.execute(QueryStr).fetchall(),


Pipes updated successfully


/Users/saijagadeeshyadavalli/Coding/Particles/GasFundies/enbridgeGitDump/src/artifacts/ModelKeeper.py:60: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(data=con.execute(QueryStr).fetchall(),


In [5]:
dfP = df.to_pandas()

In [6]:
dfP

,GFLOC,LocPurpDesc,Loc,LocName,LocZn,IT,FlowInd,LocSegment,DesignCapacity,OperatingCapacity,TotalScheduledQuantity,OperationallyAvailableCapacity,AllQtyAvail,QtyReason,Timestamp,TSP,EffGasDayTime,CycleDesc
0,114OA000001D,Delivery Location,00001,"Partroy (MORRIS,NJ)",None,N,D,None,18022.0,18022.0,5000.0,13022.0,Y,None,2025-11-18 20:46:24.049129,6951446,2025-11-15,INTRDY_2025-11-16_0900
1,114OA000002D,Delivery Location,00002,"Waterbury (NEW HAVEN,CT)",None,N,D,None,208442.0,216006.0,13109.0,202897.0,Y,None,2025-11-18 20:46:24.049129,6951446,2025-11-15,INTRDY_2025-11-16_0900
2,114OA000003D,Delivery Location,00003,North Haven (NEW HAVEN),None,N,D,None,342081.0,342081.0,62074.0,280007.0,Y,None,2025-11-18 20:46:24.049129,6951446,2025-11-15,INTRDY_2025-11-16_0900
3,114OA000004D,Delivery Location,00004,"Rhode Island Energy -DEY ST. (PROVIDENCE,RI)",None,N,D,None,157437.0,157437.0,35839.0,121598.0,Y,None,2025-11-18 20:46:24.049129,6951446,2025-11-15,INTRDY_2025-11-16_0900
4,114OA000005D,Delivery Location,00005,"Willimantic (NEW HAVEN, CT)",None,N,D,None,15880.0,15880.0,4018.0,11862.0,Y,None,2025-11-18 20:46:24.049129,6951446,2025-11-15,INTRDY_2025-11-16_0900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799,106OA045319D,Delivery Location,45319,TENNESSEE GAS MAINLINE (TPGS1030B),None,N,D,None,712396.0,712396.0,19001.0,693395.0,Y,None,2025-11-18 20:46:24.049129,791204600,2025-11-16,INTRDYC_2025-11-16_2030
2800,106OA045321R,Receipt Location,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,Y,R,None,287472.0,309861.0,10844.0,299017.0,Y,None,2025-11-18 20:46:24.049129,791204600,2025-11-16,INTRDYC_2025-11-16_2030
2801,106OA045321D,Delivery Location,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,N,D,None,407083.0,407083.0,18683.0,388400.0,Y,None,2025-11-18 20:46:24.049129,791204600,2025-11-16,INTRDYC_2025-11-16_2030
2802,106OA045323D,Delivery Location,45323,MATTERHORN PIPELINE,None,N,D,None,508854.0,508854.0,0.0,508854.0,Y,None,2025-11-18 20:46:24.049129,791204600,2025-11-16,INTRDYC_2025-11-16_2030


In [11]:
dfP

,GFLOC,LocPurpDesc,Loc,LocName,LocZn,IT,FlowInd,LocSegment,DesignCapacity,OperatingCapacity,TotalScheduledQuantity,OperationallyAvailableCapacity,AllQtyAvail,QtyReason,Timestamp,TSP,EffGasDayTime,CycleDesc
0,114OA000001D,Delivery Location,00001,"Partroy (MORRIS,NJ)",None,N,D,None,18022.0,18022.0,5000.0,13022.0,Y,None,2025-11-18 20:46:24.049129,6951446,2025-11-15,INTRDY_2025-11-16_0900
1,114OA000002D,Delivery Location,00002,"Waterbury (NEW HAVEN,CT)",None,N,D,None,208442.0,216006.0,13109.0,202897.0,Y,None,2025-11-18 20:46:24.049129,6951446,2025-11-15,INTRDY_2025-11-16_0900
2,114OA000003D,Delivery Location,00003,North Haven (NEW HAVEN),None,N,D,None,342081.0,342081.0,62074.0,280007.0,Y,None,2025-11-18 20:46:24.049129,6951446,2025-11-15,INTRDY_2025-11-16_0900
3,114OA000004D,Delivery Location,00004,"Rhode Island Energy -DEY ST. (PROVIDENCE,RI)",None,N,D,None,157437.0,157437.0,35839.0,121598.0,Y,None,2025-11-18 20:46:24.049129,6951446,2025-11-15,INTRDY_2025-11-16_0900
4,114OA000005D,Delivery Location,00005,"Willimantic (NEW HAVEN, CT)",None,N,D,None,15880.0,15880.0,4018.0,11862.0,Y,None,2025-11-18 20:46:24.049129,6951446,2025-11-15,INTRDY_2025-11-16_0900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799,106OA045319D,Delivery Location,45319,TENNESSEE GAS MAINLINE (TPGS1030B),None,N,D,None,712396.0,712396.0,19001.0,693395.0,Y,None,2025-11-18 20:46:24.049129,791204600,2025-11-16,INTRDYC_2025-11-16_2030
2800,106OA045321R,Receipt Location,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,Y,R,None,287472.0,309861.0,10844.0,299017.0,Y,None,2025-11-18 20:46:24.049129,791204600,2025-11-16,INTRDYC_2025-11-16_2030
2801,106OA045321D,Delivery Location,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,N,D,None,407083.0,407083.0,18683.0,388400.0,Y,None,2025-11-18 20:46:24.049129,791204600,2025-11-16,INTRDYC_2025-11-16_2030
2802,106OA045323D,Delivery Location,45323,MATTERHORN PIPELINE,None,N,D,None,508854.0,508854.0,0.0,508854.0,Y,None,2025-11-18 20:46:24.049129,791204600,2025-11-16,INTRDYC_2025-11-16_2030


In [15]:
dfP[dfP[['GFLOC', 'CycleDesc']].duplicated()]
# .drop_duplicates(subset=['GFLOC', 'CycleDesc'], keep='first')

,GFLOC,LocPurpDesc,Loc,LocName,LocZn,IT,FlowInd,LocSegment,DesignCapacity,OperatingCapacity,TotalScheduledQuantity,OperationallyAvailableCapacity,AllQtyAvail,QtyReason,Timestamp,TSP,EffGasDayTime,CycleDesc
1533,112OA072770D,Receipt Location,72770,PEPL/TETCO (LEBANON LATERAL-TETCO CAPACITY),2,Y,D,None,425000.0,425000.0,-349.0,425000.0,Y,None,2025-11-18 20:46:24.049129,7932908,2025-11-15,INTRDY_2025-11-16_0900
2268,112OA072770D,Receipt Location,72770,PEPL/TETCO (LEBANON LATERAL-TETCO CAPACITY),2,Y,D,None,425000.0,425000.0,-349.0,425000.0,Y,None,2025-11-18 20:46:24.049129,7932908,2025-11-16,INTRDYC_2025-11-16_2030


In [2]:
dfPipes = getPipes('Enbridge')

/Users/saijagadeeshyadavalli/Coding/Particles/GasFundies/enbridgeGitDump/src/artifacts/ModelKeeper.py:60: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(data=con.execute(QueryStr).fetchall(),


In [10]:
def paddedString(inSeries: pl.Series) -> pl.Series:
    return pl.Series(map(lambda inString: str(inString).rjust(6, '0'), inSeries))

df.join(dfPipes[['GFPipeID','TSP']],on='TSP',how='inner')\
    .with_columns(
        pl.col('GFPipeID').cast(pl.String),
        pl.col('Loc').map_batches(paddedString,return_dtype=pl.String)
    )\
    .with_columns(
        pl.concat_str([pl.col('GFPipeID'),pl.lit("OA"),pl.col('Loc'),pl.col('FlowInd')],separator='').alias('GFLOC')
    ).select('GFLOC')

GFLOC
str
"""114OA000001D"""
"""114OA000002D"""
"""114OA000003D"""
"""114OA000004D"""
"""114OA000005D"""
…
"""106OA045319D"""
"""106OA045321R"""
"""106OA045321D"""


12

In [ ]:
df[['TSP', 'TSP Name']].drop_duplicates()

,TSP,TSP Name
0,835460478,"Egan Hub Storage, LLC"
0,966981599,"Garden Banks Gas Pipeline, LLC"
0,80151107,"NEXUS Gas Transmission, LLC (U.S.)"
0,7921323,"East Tennessee Natural Gas, LLC"
0,827593182,"Steckman Ridge, LP"
0,81787256,"Maritimes & Northeast Pipeline, L.L.C. (U.S.)"
0,791204600,Tres Palacios Gas Storage LLC
0,791287274,Saltville Gas Storage Company L.L.C.
0,968195714,"Big Sandy Pipeline, LLC"
0,614834559,Bobcat Gas Storage


In [ ]:
'TSP Name' in df.columns

True

In [ ]:
Path('./downloads/enbridge/OA').resolve()

PosixPath('/Users/saijagadeeshyadavalli/Coding/Particles/GasFundies/enbridgeGitDump/downloads/enbridge/OA')

In [ ]:
dfOA = formatOA()
# .to_pandas()

In [ ]:
dfOA

,EffGasDayTime,TSP_Name,CycleDesc,LocPurpDesc,Loc,LocName,LocZn,LocSegment,DesignCapacity,OperatingCapacity,TotalScheduledQuantity,OperationallyAvailableCapacity,IT,FlowInd,AllQtyAvail,QtyReason,Timestamp,TSP
0,2025-11-15,ALGONQUIN,INTRDY_2025-11-16_0900,Delivery Location,00001,"Partroy (MORRIS,NJ)",None,None,18022.0,18022.0,5000.0,13022.0,N,D,Y,None,2025-11-18 18:49:34.462497,6951446
1,2025-11-15,ALGONQUIN,INTRDY_2025-11-16_0900,Delivery Location,00002,"Waterbury (NEW HAVEN,CT)",None,None,208442.0,216006.0,13109.0,202897.0,N,D,Y,None,2025-11-18 18:49:34.462497,6951446
2,2025-11-15,ALGONQUIN,INTRDY_2025-11-16_0900,Delivery Location,00003,North Haven (NEW HAVEN),None,None,342081.0,342081.0,62074.0,280007.0,N,D,Y,None,2025-11-18 18:49:34.462497,6951446
3,2025-11-15,ALGONQUIN,INTRDY_2025-11-16_0900,Delivery Location,00004,"Rhode Island Energy -DEY ST. (PROVIDENCE,RI)",None,None,157437.0,157437.0,35839.0,121598.0,N,D,Y,None,2025-11-18 18:49:34.462497,6951446
4,2025-11-15,ALGONQUIN,INTRDY_2025-11-16_0900,Delivery Location,00005,"Willimantic (NEW HAVEN, CT)",None,None,15880.0,15880.0,4018.0,11862.0,N,D,Y,None,2025-11-18 18:49:34.462497,6951446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799,2025-11-16,TRES PALACIOS,INTRDYC_2025-11-16_2030,Delivery Location,45319,TENNESSEE GAS MAINLINE (TPGS1030B),None,None,712396.0,712396.0,19001.0,693395.0,N,D,Y,None,2025-11-18 18:49:34.462497,791204600
2800,2025-11-16,TRES PALACIOS,INTRDYC_2025-11-16_2030,Receipt Location,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,None,287472.0,309861.0,10844.0,299017.0,Y,R,Y,None,2025-11-18 18:49:34.462497,791204600
2801,2025-11-16,TRES PALACIOS,INTRDYC_2025-11-16_2030,Delivery Location,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,None,407083.0,407083.0,18683.0,388400.0,N,D,Y,None,2025-11-18 18:49:34.462497,791204600
2802,2025-11-16,TRES PALACIOS,INTRDYC_2025-11-16_2030,Delivery Location,45323,MATTERHORN PIPELINE,None,None,508854.0,508854.0,0.0,508854.0,N,D,Y,None,2025-11-18 18:49:34.462497,791204600


In [ ]:
dfOA[~dfOA[['TSP_Name', 'TSP']].duplicated()][['TSP_Name', 'TSP']]

,TSP_Name,TSP
0,ALGONQUIN,6951446
392,BOBCAT,614834559
412,BSP,968195714
416,EGAN,835460478
452,EAST TENN,7921323
862,GARDEN BANKS,966981599
890,GPL,36638044
900,MISSISSIPPI CANYON,946822749
916,M&N US,81787256
972,NAUTILUS,176824738


In [ ]:
pipesLF=pl.LazyFrame(getPipes('Enbridge'))

In [ ]:
dfOA = pl.LazyFrame(formatOA())

In [ ]:
joinset=set(dfOA.join(pipesLF, on='TSP', how='inner').collect().to_pandas()['TSP'].unique())

In [ ]:
OAset = set(formatOA().to_pandas()['TSP'].unique())

In [ ]:
OAset - joinset

{np.int64(36638044), np.int64(612195573), np.int64(946822749)}

In [ ]:
dfOA['']

,EffGasDayTime,CycleDesc,LocPurpDesc,Loc,LocName,LocZn,LocSegment,DesignCapacity,OperatingCapacity,TotalScheduledQuantity,OperationallyAvailableCapacity,IT,FlowInd,AllQtyAvail,QtyReason,Timestamp,TSP
0,2025-11-15,INTRDY_2025-11-16_0900,Delivery Location,00001,"Partroy (MORRIS,NJ)",None,None,18022.0,18022.0,5000.0,13022.0,N,D,Y,None,2025-11-18 18:44:13.543159,6951446
1,2025-11-15,INTRDY_2025-11-16_0900,Delivery Location,00002,"Waterbury (NEW HAVEN,CT)",None,None,208442.0,216006.0,13109.0,202897.0,N,D,Y,None,2025-11-18 18:44:13.543159,6951446
2,2025-11-15,INTRDY_2025-11-16_0900,Delivery Location,00003,North Haven (NEW HAVEN),None,None,342081.0,342081.0,62074.0,280007.0,N,D,Y,None,2025-11-18 18:44:13.543159,6951446
3,2025-11-15,INTRDY_2025-11-16_0900,Delivery Location,00004,"Rhode Island Energy -DEY ST. (PROVIDENCE,RI)",None,None,157437.0,157437.0,35839.0,121598.0,N,D,Y,None,2025-11-18 18:44:13.543159,6951446
4,2025-11-15,INTRDY_2025-11-16_0900,Delivery Location,00005,"Willimantic (NEW HAVEN, CT)",None,None,15880.0,15880.0,4018.0,11862.0,N,D,Y,None,2025-11-18 18:44:13.543159,6951446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799,2025-11-16,INTRDYC_2025-11-16_2030,Delivery Location,45319,TENNESSEE GAS MAINLINE (TPGS1030B),None,None,712396.0,712396.0,19001.0,693395.0,N,D,Y,None,2025-11-18 18:44:13.543159,791204600
2800,2025-11-16,INTRDYC_2025-11-16_2030,Receipt Location,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,None,287472.0,309861.0,10844.0,299017.0,Y,R,Y,None,2025-11-18 18:44:13.543159,791204600
2801,2025-11-16,INTRDYC_2025-11-16_2030,Delivery Location,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,None,407083.0,407083.0,18683.0,388400.0,N,D,Y,None,2025-11-18 18:44:13.543159,791204600
2802,2025-11-16,INTRDYC_2025-11-16_2030,Delivery Location,45323,MATTERHORN PIPELINE,None,None,508854.0,508854.0,0.0,508854.0,N,D,Y,None,2025-11-18 18:44:13.543159,791204600


In [ ]:
dfOA.collect().to_pandas()

# ['TSP'].unique()

,Cycle_Desc,Eff_Gas_Day,Loc,Loc_Name,Loc_Zn,Flow_Ind_Desc,Loc_Purp_Desc,IT,All_Qty_Avail,Total_Design_Capacity,Operating_Capacity,Total_Scheduled_Quantity,Operationally_Available_Capacity,TSP
0,INTRDY_2025-11-16_0900,11-15-2025,00001,"Partroy (MORRIS,NJ)",None,Delivery,Delivery Location,N,Y,"18,022","18,022","5,000","13,022",006951446
1,INTRDY_2025-11-16_0900,11-15-2025,00002,"Waterbury (NEW HAVEN,CT)",None,Delivery,Delivery Location,N,Y,"208,442","216,006","13,109","202,897",006951446
2,INTRDY_2025-11-16_0900,11-15-2025,00003,North Haven (NEW HAVEN),None,Delivery,Delivery Location,N,Y,"342,081","342,081","62,074","280,007",006951446
3,INTRDY_2025-11-16_0900,11-15-2025,00004,"Rhode Island Energy -DEY ST. (PROVIDENCE,RI)",None,Delivery,Delivery Location,N,Y,"157,437","157,437","35,839","121,598",006951446
4,INTRDY_2025-11-16_0900,11-15-2025,00005,"Willimantic (NEW HAVEN, CT)",None,Delivery,Delivery Location,N,Y,"15,880","15,880","4,018","11,862",006951446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799,INTRDYC_2025-11-16_2030,11-16-2025,45319,TENNESSEE GAS MAINLINE (TPGS1030B),None,Delivery,Delivery Location,N,Y,"712,396","712,396","19,001","693,395",791204600
2800,INTRDYC_2025-11-16_2030,11-16-2025,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,Receipt,Receipt Location,Y,Y,"287,472","309,861","10,844","299,017",791204600
2801,INTRDYC_2025-11-16_2030,11-16-2025,45321,ENTERPRISE TEXAS (VALERO) (TPGS1020B),None,Delivery,Delivery Location,N,Y,"407,083","407,083","18,683","388,400",791204600
2802,INTRDYC_2025-11-16_2030,11-16-2025,45323,MATTERHORN PIPELINE,None,Delivery,Delivery Location,N,Y,"508,854","508,854",0,"508,854",791204600


In [ ]:
selectedCols = ['Cycle_Desc', 'Eff_Gas_Day', 'Loc', 'Loc_Name', 'Loc_Zn', 'Flow_Ind_Desc', 'Loc_Purp_Desc', 'IT', 'All_Qty_Avail',
                'Total_Design_Capacity', 'Operating_Capacity', 'Total_Scheduled_Quantity', 'Operationally_Available_Capacity']


In [ ]:
df = pl.scan_csv(Path('./downloads/enbridge/OA') / "*_OA_*.csv",
                    glob=True,
                    has_header=True,
                    )\
    .filter(
    ~functools.reduce(
        operator.and_,
        map(lambda col: pl.col(col).is_null(), selectedCols)
    ))


In [ ]:
df=df.collect().to_pandas()

In [ ]:
df[df['TSP'].isin(['612195573','946822749','36638044'])]['Loc_Name'].unique()

array(['PEPL MAUMEE', 'ANR FALLEN TIMBERS', 'OREGON CLEAN ENERGY PLANT',
       'Interstate Chemical', 'CLEVELAND CLIFFS', 'OLYMPUS (992003)',
       'VENICE GAS PLANT (992002)', 'VENICE GAS PLANT BYPASS (992010)',
       'KINETICA ENERGY EXPRESS (992009)',
       'HIGH POINT GAS TRANSMISSION (992006)', 'Ursa Platform MC 809-2',
       'MC 989 Vito', 'MARS PLATFORM (R-M0022 D-M0024)',
       'EARLY GROVE REC/INJ (59147)', 'SALTVILLE REC INTO P-25',
       'EARLY GROVE DEL/WD (59009)',
       'United Salt - Evaporator Fuel Gas - Smyth Co., VA',
       'SALTVILLE LLC STORAGE INJECTION', 'SALTVILLE STORAGE WITHDRAWAL',
       'SALTVILLE LLC STORAGE WITHDRAWAL', 'SALTVILLE STORAGE INJECTION'],
      dtype=object)

In [ ]:
len(df[~df.isnull().any(axis=1)])

In [ ]:
df[df[selectedCols].isnull().any(axis=1)]

In [ ]:
dfOA.join(pipesLF, on='TSP', how='outer')